# Taichi
## Description
Taichi est un langage de programmation intégré à python qui permet d'écrire du code hautement performant en utilisant le parrallélisme et la vec2isation. Il est principalement utilisé pour la simulation physique, la rendu d'image et l'apprentissage profond. Taichi est conçu pour être facile à utiliser et à apprendre, il est également très flexible et extensible. Il est open source et est disponible sur github.

## Installation
Pour installer taichi, il suffit d'exécuter la commande suivante:
```bash
$ pip install taichi -U
```

In [3]:

!pip install taichi -U

Defaulting to user installation because normal site-packages is not writeable


## Prise en main

Pour prendre en main taichi, nous allons écrire un programme simple qui affiche une fractale à l'écran.

## Version non optimisée du Labo 7

In [14]:
def find_sequence_in_file(filename, sequence):
    try:
        with open(filename, 'r') as file:
            sequence_length = len(sequence)
            match_count = 0

            while True:
                ch = file.read(1)
                if not ch:
                    break

                if ch == sequence[match_count]:
                    match_count += 1
                    if match_count == sequence_length:
                        return True
                else:
                    match_count = 1 if ch == sequence[0] else 0

    except FileNotFoundError:
        print(f"Could not open file {filename}", file=sys.stderr)
        return False

    return False

filename = "/home/miguel/Bureau/HPC/labos_HPC/lab7/code/inputs/10m.txt"
sequence = "20058243"

found = find_sequence_in_file(filename, sequence)
if found:
    print("Sequence found")
else:
    print("Sequence not found")

Sequence found


## Version optimisiée avec Taichi

In [31]:
import taichi as ti
import numpy as np
import os

# Initialiser Taichi pour utiliser le CPU (ou le GPU si disponible)
ti.init(arch=ti.cpu)

# Définir la séquence à rechercher
sequence = "20058243"
sequence_length = len(sequence)
sequence_np = np.array([ord(c) for c in sequence], dtype=np.int32)

# créer un tableau Taichi pour stocker la séquence
seq = ti.field(dtype=ti.int32, shape=sequence_length)

print(type(seq))

# Fonction pour lire le fichier et le diviser en morceaux
def read_file_chunks(filename, chunk_size=1024):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Could not open file {filename}")
    
    with open(filename, 'r') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            yield chunk

# Kernel Taichi pour rechercher la séquence dans un morceau
@ti.kernel
def search_sequence_in_chunk(chunk: ti.lang.field.ScalarField, seq: ti.ndarray, result: ti.ndarray):
    chunk_length = chunk.shape[0]
    for i in range(chunk_length):
        match_count = 0
        for j in range(seq.shape[0]):
            if i + j < chunk_length and chunk[i + j] == seq[j]:
                match_count += 1
            else:
                break
        if match_count == seq.shape[0]:
            result[0] = 1  # Séquence trouvée

def find_sequence_in_file_taichi(filename, sequence):
    chunk_size = 1024
    result = ti.field(dtype=ti.int32, shape=())
    
    for chunk in read_file_chunks(filename, chunk_size):
        chunk_np = np.array([ord(c) for c in chunk], dtype=np.int32)
        search_sequence_in_chunk(chunk_np, sequence_np, result)
        
        if result[0] == 1:
            return True
    
    return False

filename = "/home/miguel/Bureau/HPC/labos_HPC/lab7/code/inputs/10m.txt"

found = find_sequence_in_file_taichi(filename, sequence)
if found:
    print("sequence found")
else:
    print("sequence not found")


[Taichi] Starting on arch=x64
<class 'taichi.lang.field.ScalarField'>


TaichiSyntaxError: Invalid type annotation (argument 0) of Taichi kernel: <function ndarray at 0x7d1fd00e7910> (<string>)